<a href="https://colab.research.google.com/github/EricRoh-kr/st9_youtubevid/blob/main/st9_youtubevid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 필요한 라이브러리 설치 및 로드
import subprocess, sys, os, zipfile, shutil, glob
from datetime import datetime
from IPython.display import display, FileLink, clear_output
import ipywidgets as widgets

# Colab 감지
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 실행 잠금
IS_RUNNING = False

def install_ytdlp():
    try:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-U", "yt-dlp"],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )
        return True
    except subprocess.CalledProcessError:
        return False

def download_videos(urls, fmt, folder, out):
    with out:
        print(f"⬇️ {len(urls)}개 영상 다운로드 중...")
        for i,u in enumerate(urls,1):
            print(f"[{i}/{len(urls)}] {u}", end=" → ")
            cmd = ["yt-dlp","-f",fmt,
                   "-o",os.path.join(folder,"%(title)s.%(ext)s"),u]
            res = subprocess.run(cmd, capture_output=True, text=True)
            print("✅" if res.returncode==0 else "❌")

def make_zip_and_cleanup(folder, out):
    # 1) 기존 ZIP 전부 삭제
    for p in glob.glob("*_유튜브다운로드.zip"):
        try: os.remove(p)
        except: pass

    # 2) 새 ZIP 생성
    now = datetime.now().strftime("%m%d_%H%M")  # 날짜만
    zip_name = f"{now}_유튜브다운로드.zip"
    with out:
        print("💾 ZIP 생성 중...")
    with zipfile.ZipFile(zip_name,"w",zipfile.ZIP_DEFLATED) as zf:
        for root,_,files in os.walk(folder):
            for f in files:
                full = os.path.join(root,f)
                zf.write(full, os.path.relpath(full,folder))
    return zip_name

# UI 준비
urls_input = widgets.Textarea(
    placeholder="유튜브 링크를 한 줄씩 입력하세요",
    layout=widgets.Layout(width="90%", height="120px")
)
quality_choice = widgets.RadioButtons(
    options=["일반화질 (720p 이하)","고화질 (최고 품질)"],
    description="화질:", style={'description_width':'initial'}
)
button = widgets.Button(
    description="▶ 영상 다운로드", button_style="success",
    layout=widgets.Layout(width="200px",height="40px")
)
out = widgets.Output(layout=widgets.Layout(width="90%",border="1px solid #ddd",padding="10px"))

def on_click(b):
    global IS_RUNNING
    if IS_RUNNING: return
    IS_RUNNING = True
    button.disabled = True
    out.clear_output()

    with out:
        # 1) 링크 체크
        urls = [u.strip() for u in urls_input.value.splitlines() if u.strip()]
        if not urls:
            print("❌ 링크를 입력해주세요."); IS_RUNNING=False; button.disabled=False; return

        # 2) yt-dlp 설치
        print("🚀 yt-dlp 설치 확인...")
        if not install_ytdlp():
            print("❌ 설치 실패"); IS_RUNNING=False; button.disabled=False; return

        # 3) 폴더 초기화
        folder = "영상저장함"
        if os.path.exists(folder): shutil.rmtree(folder)
        os.makedirs(folder)

        # 4) 영상 다운로드
        fmt = "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best" \
              if quality_choice.value.startswith("고화질") \
              else "best[height<=720]/best"
        download_videos(urls, fmt, folder, out)

        # 5) ZIP 생성 & 이전 ZIP 정리
        zip_name = make_zip_and_cleanup(folder, out)
        print(f"✅ ZIP 완성: {zip_name}")

        # 6) 다운로드 혹은 링크
        if IN_COLAB:
            files.download(zip_name)
        else:
            print("👉 링크 클릭 후 다운로드:")
            display(FileLink(zip_name))

    IS_RUNNING = False
    button.disabled = False

button.on_click(on_click)

clear_output(wait=True)
display(widgets.VBox([
    widgets.HTML("<h2>🎬 전략 9팀 YouTube 다운로더</h2>"),
    urls_input, quality_choice, button,
    widgets.HTML("<hr><h3>진행 상황</h3>"), out
]))
